第六章 使用一等函数实现设计模式

![chapter6-1](image/chapter6-1.png)

In [15]:
#实现order类，支持插入式折扣策略
from abc import ABC,abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    
    def __init__(self,customer,cart,promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion=promotion
        
    
    def total(self):
        if not hasattr(self,'__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount =0
        else:
            discount=self.promotion.discount(self)
        return self.total()-discount
    
    def __repr__(self):
        fmt='<Order total:{:.2f} due: {:.2f}>'
        return fmt.format(self.total(),self.due())
    
class Promotion(ABC):
    @abstractmethod
    def discount(self,order):
        '''返回折扣金额（正值)'''

class FidelityPromo(Promotion):
    '''为积分为1000及以上的顾客提供5%折扣'''
    
    def discount(self,order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    '''单个商品为20个或以上提供10%折扣'''
    def discount(self,order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total()*0.1
        return discount

class LargeOrderPromo(Promotion):
    '''订单中不同商品达到十个或以上提供7%折扣'''
    def discount(self,order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >=10:
            return  order.total() * 0.07
        return 0
        
        

In [16]:
joe = Customer('John Doe',0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),LineItem('apple', 10, 1.5),LineItem('watermellon', 5, 5.0)]
Order(joe,cart,FidelityPromo())

<Order total:42.00 due: 42.00>

In [17]:
Order(ann,cart,FidelityPromo())

<Order total:42.00 due: 39.90>

In [18]:
banana_cart = [LineItem('banana', 30, .5),LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo()) 

<Order total:30.00 due: 28.50>

In [19]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe,long_order,LargeOrderPromo())

<Order total:10.00 due: 9.30>

6.1.2 使用函数实现“策略”模式

In [16]:

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity
    
class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # <1>
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())



def fidelity_promo(order):  # <3>
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

promos = [fidelity_promo,bulk_item_promo,large_order_promo]

def best_promo(order):
    '''选择最佳折扣'''
    return max(promo(order) for promo in promos)


In [4]:
joe = Customer('John Doe', 0)

In [5]:
ann = Customer('Ann Smith', 1100)

In [6]:
cart = [LineItem('banana', 4, .5),LineItem('apple', 10, 1.5),LineItem('watermellon', 5, 5.0)]

In [7]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [8]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [10]:
banana_cart = [LineItem('banana', 30, .5),LineItem('apple', 10, 1.5)]

In [11]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [12]:
Order(joe, banana_cart, bulk_item_promo) 

<Order total: 30.00 due: 28.50>

In [13]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [14]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [15]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

In [17]:
Order(joe,long_order,best_promo)

<Order total: 10.00 due: 9.30>

In [18]:
Order(joe,banana_cart,best_promo)

<Order total: 30.00 due: 28.50>

In [19]:
Order(ann,cart,best_promo)

<Order total: 42.00 due: 39.90>

注意：若想添加新的促销策略，要定义相应的函数，还要记得把他添加到promos列表中

6.1.4 找出模块中的全部策略

在python中，模块也是一等对象，而且标准库提供了几个处理模块的函数。

global()返回一个字典，表示当前的全局符号表。这个符号表始终针对当前模块（对函数或方法来说，是指定义它们的模块，而不是调用它们的模块）

In [23]:
#globals()

In [21]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

In [22]:
promos

[<function __main__.large_order_promo>,
 <function __main__.fidelity_promo>,
 <function __main__.bulk_item_promo>]

In [24]:
def best_promo(order):
    '''选择最佳折扣'''
    return max(promo(order) for promo in promos)

添加新的促销策略时，promos会自动添加，不用重写代码

In [ ]:
#内省单独的promotions模块，构建promos列表
import inspect
import promotions
promos = [func for name,func in inspect.getmembers(promotions,inspect.isfunction)]

def best_promo(order):
    return max(promo(order) for promo in promos)

这里需导入promotions模块和高阶内省函数的inspect模块。inspe.getmembers函数用于获取对象（这里是promotions模块）的属性，第二个参数是可选的判断条件（一个布尔值函数）。我们使用的是inspe.isfunction。